# Imports

In [1]:
import decaylanguage as dl

from particle.pdgid import is_baryon, is_lepton, is_quark, is_diquark

from particle.converters.bimap import DirectionalMaps
EvtGen2PDGID, _ = DirectionalMaps("EvtGenName", "PDGID")
_EvtGen2LatexNameMap, _ = DirectionalMaps("EvtGenName", "LaTexName")

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

import json

# Definitions

In [2]:
def get_stable(parser, additional_stable):
    list_of_lists = parser.list_decay_modes('B+')
    flattened_set = set([val for sublist in list_of_lists for val in sublist])

    consider_as_stable = []

    print('\n')

    for i in flattened_set:
        try:
            if is_baryon(EvtGen2PDGID[i]):
                consider_as_stable.append(i)
            elif is_lepton(EvtGen2PDGID[i]):
                consider_as_stable.append(i)
            elif is_quark(EvtGen2PDGID[i]):
                consider_as_stable.append(i)
            elif is_diquark(EvtGen2PDGID[i]):
                consider_as_stable.append(i)
        except:
            print(f'Not sure about {i}, not considering as stable.')
            continue
            
    for i in additional_stable:
        consider_as_stable.append(i)
        
    consider_as_stable = tuple(consider_as_stable)

    print('\n')

    return consider_as_stable

In [3]:
def convert_str_to_latex(string):
    daughters = string.split()
    daughters_latex = []
    for i in daughters:
        if i == '-->':
            daughters_latex.append('\\rightarrow')
        elif i == ';':
            daughters_latex.append('$; $')
        elif i == 'gammaF':
            daughters_latex.append('\\gamma^{F}')
        elif i == 'f_0(600)':
            daughters_latex.append('f_0 (600)')
        else:
            try:
                daughters_latex.append(_EvtGen2LatexNameMap[i])
            except:
                print(f'Not found: {i}')
                daughters_latex.append(i)

    return('$' + ' '.join(daughters_latex) + '$')

In [4]:
def build_table(mode, b_parser, b2_parser, mother='B+', min_con=0.01):
    # Build Exclusive table
    b_ex = b_parser.build_decay_chains_to_specific_fsp_new(mother, stable_particles=b_consider_as_stable,
                                                            fsp=mode)
    b2_ex = b2_parser.build_decay_chains_to_specific_fsp_new(mother, stable_particles=b2_consider_as_stable,
                                                            fsp=mode)
    b_df = dl.DecayChainToTable(b_ex).table.sort_values(by='BF', ascending=False, ignore_index=True)
    b2_df = dl.DecayChainToTable(b2_ex).table.sort_values(by='BF', ascending=False, ignore_index=True)

    b_df['BF'] = b_df['BF'] * 1e2
    b2_df['BF'] = b2_df['BF'] * 1e2

    b_df.columns = b_df.columns.str.replace('BF', 'Belle BF')
    b2_df.columns = b2_df.columns.str.replace('BF', 'Belle II BF')

    # Merge Belle and Belle II DataFrames
    df = b_df.merge(b2_df, on='Decay', how='outer')
    df['Decay'] = df.apply(lambda x: convert_str_to_latex(x.Decay), axis=1)
    df['Belle_con'] = df.apply(lambda x: x['Belle BF']/df['Belle BF'].sum(), axis=1)
    df['Belle_II_con'] = df.apply(lambda x: x['Belle II BF']/df['Belle BF'].sum(), axis=1)

    # Convert to LaTeX
    min_con_cut = f'(Belle_con > {min_con}) or (Belle_II_con > {min_con})'
    df_latex = df.query(min_con_cut)[['Decay', 'Belle BF', 'Belle II BF']]
    df_latex = df_latex.append({'Decay': 'Rest of Exclusive', 
                            'Belle BF': df.query(min_con_cut)['Belle BF'].sum(), 
                            'Belle II BF': df.query(min_con_cut)['Belle II BF'].sum()}, ignore_index=True)
    df_latex = df_latex.append({'Decay': 'Sum of Exclusive', 'Belle BF':df['Belle BF'].sum(), 'Belle II BF':df['Belle II BF'].sum()}, ignore_index=True)

    # Build PYTHIA table
    b_p = b_parser.build_decay_chains_to_specific_fsp_pythia(mother_decs=pythia_b['B+'], stable_particles=b_consider_as_stable,
                                                                fsp=mode)
    b2_p = b2_parser.build_decay_chains_to_specific_fsp_pythia(mother_decs=pythia_b2['B+'], stable_particles=b2_consider_as_stable,
                                                                fsp=mode)
    try:
        b_p_df = dl.DecayChainToTable(b_p).table.sort_values(by='BF', ascending=False, ignore_index=True)
    except:
        b_p_df = pd.DataFrame(columns=['Decay', 'BF'])

    try:
        b2_p_df = dl.DecayChainToTable(b2_p).table.sort_values(by='BF', ascending=False, ignore_index=True)
    except:
        b2_p_df = pd.DataFrame(columns=['Decay', 'BF'])

    b_p_df['BF'] = b_p_df['BF'] * 1e2
    b2_p_df['BF'] = b2_p_df['BF'] * 1e2

    b_p_df.columns = b_p_df.columns.str.replace('BF', 'Belle BF')
    b2_p_df.columns = b2_p_df.columns.str.replace('BF', 'Belle II BF')

    # Merge PYTHIA tables
    df_p = b_p_df.merge(b2_p_df, on='Decay', how='outer')
    if len(df_p)>0:
        df_p['Decay'] = df_p.apply(lambda x: '\color{blue}{'+convert_str_to_latex(x.Decay)+'}', axis=1)
        df_p['Belle_con'] = df_p.apply(lambda x: x['Belle BF']/df_p['Belle BF'].sum(), axis=1)
        df_p['Belle_II_con'] = df_p.apply(lambda x: x['Belle II BF']/df_p['Belle BF'].sum(), axis=1)
    else:
        df_p['Belle_con'] = ''
        df_p['Belle_II_con'] = ''

    # Convert PYTHIA table to LaTeX
    df_p_latex = df_p.query(min_con_cut)[['Decay', 'Belle BF', 'Belle II BF']]
    df_p_latex = df_p_latex.append({'Decay': '\color{blue}{Rest of PYTHIA}', 
                                'Belle BF': df_p.query(min_con_cut)['Belle BF'].sum(), 
                                'Belle II BF': df_p.query(min_con_cut)['Belle II BF'].sum()}, ignore_index=True)
    df_p_latex = df_p_latex.append({'Decay': '\color{blue}{Sum of PYTHIA}', 'Belle BF':df_p['Belle BF'].sum(), 'Belle II BF':df_p['Belle II BF'].sum()}, ignore_index=True)
    df_p_latex

    # Merge Exclusive and PYTHIA tables
    df_grand_latex = df_latex.append(df_p_latex)
    df_grand_latex = df_grand_latex.append({'Decay': 'Total Sum',
                                            'Belle BF':df['Belle BF'].sum()+df_p['Belle BF'].sum(),
                                            'Belle II BF':df['Belle II BF'].sum()+df_p['Belle II BF'].sum()}, ignore_index=True)

    # Final Print
    print('\n \n The essential part: \n \n')
    print(df_grand_latex.to_latex(index=False, escape=False))
    
    return df_grand_latex

# Inputs

In [5]:
belle_DEC = 'DECAY.DEC'
belle2_DEC = 'DECAY_BELLE2.DEC'

belle_pythia = 'Bp_pythia_b.json'
belle2_pythia = 'Bp_pythia_b2.json'

In [6]:
fei_modes = {
    1: ['anti-D0', 'pi+'],
    2: ['anti-D0', 'pi+', 'pi0'],
    3: ['anti-D0', 'pi+', 'pi0', 'pi0'],
    4: ['anti-D0', 'pi+', 'pi+', 'pi-'],
    5: ['anti-D0', 'pi+', 'pi+', 'pi-', 'pi0'],
    6: ['anti-D*0', 'pi+'],
    7: ['anti-D*0', 'pi+', 'pi0'],
    8: ['anti-D*0', 'pi+', 'pi0', 'pi0'],
    9: ['anti-D*0', 'pi+', 'pi+', 'pi-'],
    10: ['anti-D*0', 'pi+', 'pi+', 'pi-', 'pi0'],
    11: ['D-', 'pi+', 'pi+'],
    12: ['D-', 'pi+', 'pi+', 'pi0'],
}

additional_stable = ['pi+', 'pi-', 'pi0', 'K+', 'gamma']

# Open

In [7]:
b_parser = dl.DecFileParser(belle_DEC)
b2_parser = dl.DecFileParser(belle2_DEC)

b_parser.parse()
b2_parser.parse()

b_consider_as_stable = get_stable(parser=b_parser, additional_stable=additional_stable)
b2_consider_as_stable = get_stable(parser=b2_parser, additional_stable=additional_stable)

/home/vidya/PhD/decaylanguage/src/decaylanguage/dec/dec.py:571: UserWarning: The following particle(s) is(are) redefined in the input .dec file with 'Decay': B_s0, anti-B_s0!
All but the first occurrence will be discarded/removed ...
  warnings.warn(msg)
/home/vidya/PhD/decaylanguage/src/decaylanguage/dec/dec.py:518: UserWarning: 
Corresponding 'Decay' statement for 'CDecay' statement(s) of following particle(s) not found:
D_sj(2700)+
K_0*(800)-
anti-K_0*(800)0
anti-Lambda_c(2765)-
anti-Xdu-.
Skipping creation of these charge-conjugate decay trees.
  warnings.warn(msg)




Not sure about D_sj(2700)+, not considering as stable.
Not sure about X(3872), not considering as stable.
Not sure about Y(3940), not considering as stable.








In [8]:
with open(belle2_pythia) as json_data:
     pythia_b=json_data.read()

pythia_b=json.loads(pythia_b)

with open(belle2_pythia) as json_data:
     pythia_b2=json_data.read()

pythia_b2=json.loads(pythia_b2)

# Make Tables

In [9]:
build_table(fei_modes[2], b_parser, b2_parser, mother='B+')

Total BF tabulated: 1.76%
Total BF tabulated: 1.74%

 
 The essential part: 
 

\begin{tabular}{lll}
\toprule
                                                                                                                              Decay & Belle BF & Belle II BF \\
\midrule
                                  $B^{+} \rightarrow \rho(770)^{+} \overline{D}^{0} $; $ \rho(770)^{+} \rightarrow \pi^{+} \pi^{0}$ &     1.34 &        1.34 \\
        $B^{+} \rightarrow \overline{D}^{*}(2007)^{0} \pi^{+} $; $ \overline{D}^{*}(2007)^{0} \rightarrow \overline{D}^{0} \pi^{0}$ &     0.32 &        0.32 \\
                                                                               $B^{+} \rightarrow \overline{D}^{0} \pi^{0} \pi^{+}$ &     0.05 &        0.05 \\
$B^{+} \rightarrow \overline{D}_{0}^{*}(2300)^{0} \pi^{+} $; $ \overline{D}_{0}^{*}(2300)^{0} \rightarrow \overline{D}^{0} \pi^{0}$ &     0.03 &        0.02 \\
$B^{+} \rightarrow \overline{D}_{2}^{*}(2460)^{0} \pi^{+} $; $ \overline{D}_{2}^{

,Decay,Belle BF,Belle II BF
0,$B^{+} \rightarrow \rho(770)^{+} \overline{D}^{0} $; $ \rho(770)^{+} \rightarrow \pi^{+} \pi^{0}$,1.34,1.34
1,$B^{+} \rightarrow \overline{D}^{*}(2007)^{0} \pi^{+} $; $ \overline{D}^{*}(2007)^{0} \rightarrow \overline{D}^{0} \pi^{0}$,0.32,0.32
2,$B^{+} \rightarrow \overline{D}^{0} \pi^{0} \pi^{+}$,0.05,0.05
3,$B^{+} \rightarrow \overline{D}_{0}^{*}(2300)^{0} \pi^{+} $; $ \overline{D}_{0}^{*}(2300)^{0} \rightarrow \overline{D}^{0} \pi^{0}$,0.03,0.02
4,$B^{+} \rightarrow \overline{D}_{2}^{*}(2460)^{0} \pi^{+} $; $ \overline{D}_{2}^{*}(2460)^{0} \rightarrow \overline{D}^{0} \pi^{0}$,0.02,0.01
5,Rest of Exclusive,1.76,1.74
6,Sum of Exclusive,1.76,1.74
7,\color{blue}{Rest of PYTHIA},0,0
8,\color{blue}{Sum of PYTHIA},0,0
9,Total Sum,1.76,1.74
